<img src="./img/monitoring.jpg"/>

#### setup the environment 

In [ ]:
from arcgis.gis import GIS
gis = GIS(profile="your_online_profile")
user = gis.users.me
um = gis.users

### Managing Named User Licenses and Entitlements

In [ ]:
# list all licenses in the organization
license_list = gis.admin.license.all()
l = license_list[-1]
l.assign(um.search("*")[3].username, ["cityEngine"])

In [ ]:
l.plot()

In [ ]:
test_user = um.search("*")[3]

In [ ]:
l.assign(test_user, ["cityEngine"])

In [ ]:
pro_license = gis.admin.license.get('ArcGIS Pro')
pro_license

In [ ]:
#get all users licensed for ArcGIS Pro
pro_license.all()

In [ ]:
%matplotlib inline
pro_license.plot()

In [ ]:
# Get license object's report property
pro_license.report

In [ ]:
pro_license.report.iloc[8]["Users"]

### Querying and Assigning Named Licenses

In [ ]:
pro_license.user_entitlement(test_user.username)

To revoke an app's license from a user, call the revoke() method from the corresponding License object.

In [ ]:
pro_license.revoke(username=test_user.username, entitlements="*")

In [ ]:
pro_license.user_entitlement(test_user.username)

You can assign licenses to an application and its extensions using the assign() method.

In [ ]:
pro_license.assign(username=test_user.username, entitlements=['3DAnalystN',
  'bathymetryN',
  'businessStdN',
  'dataInteropN',
  'dataReviewerN',
  'desktopAdvN',
  'geostatAnalystN',
  'imageAnalystN',
  'networkAnalystN',
  'publisherN',
  'spatialAnalystN'])

In [ ]:
pro_license.user_entitlement(test_user.username)

# Generating Reports for Organization
Generate the reports of the overall usage of the organizations.  
Reports define organization usage metrics in one place for the day, week, or month.  
Administrators can monitor who is using which services, consuming how much credits and storage within certain time period.



In [ ]:
import datetime as _dt
import pandas as pd

In [ ]:
date_time_str = '01/01/23'
then = _dt.datetime.strptime(date_time_str, '%d/%m/%y')
val = int(then.timestamp() * 1000)

In [ ]:
# Create the report, this will create an item in your organization
sun_dec10 = _dt.datetime(2023, 12, 1)
final_item = gis.users.me.report(
            report_type='content', duration="monthly", start_time=sun_dec10
    
        )
final_item

In [ ]:
# Read the item data into a temporary CSV file
csv_file = final_item.get_data()

# Read the CSV as a DataFrame
df = pd.read_csv(csv_file)

#### See the Most Viewed Items

In [ ]:
df.sort_values(by=['View Counts'], ascending=False).head()

#### See the Largest Sized Items

In [ ]:
df.sort_values(by=['File Storage Size'], ascending=False).head()[['Title','Item Type', 'File Storage Size']]

#### Understand Storage by User

In [ ]:
gb = df.groupby("Owner")['File Storage Size'].sum()
gb.nlargest(4).plot(kind='barh')

In [ ]:
gb

## Credit Reporting

In [ ]:
#credits
import datetime as _dt
date_time_str = '4/2/2024'
then = _dt.datetime.strptime(date_time_str, '%d/%m/%Y')

monthly_item = gis.users.me.report(
            report_type='credits', duration="weekly", start_time=then
        )

In [ ]:
monthly_item

In [ ]:
import pandas as pd
# Read the item data into a temporary CSV file
csv_credits_file = monthly_item.get_data()

# Read the CSV as a DataFrame
df = pd.read_csv(csv_credits_file, skiprows=3)

In [ ]:
df.head(7)

In [ ]:
q = df['GeoEnrichment'] > 0
df[q]

## Using Search Operations to Monitor Sites

- The `search` and `advanced_search` operations allows administrators to query GIS systems easily to find out information about the system
- The information can easily be parsed using other 3rd party libraries like pandas
- This information can be done multiple time a day or hour

### Monitoring Content Using Searches

#### Content Searches

Example: Finding Content Types Generated Using Searches

Whole document on advanced searching: https://doc.arcgis.com/en/arcgis-online/reference/advanced-search.htm

In [ ]:
cm = gis.content

In [ ]:
import datetime as _dt
now = _dt.datetime.now()
then = now - _dt.timedelta(days=7)
search = cm.advanced_search(
            f"orgid: {gis.properties.id} AND created: [{int(then.timestamp()* 1000)} TO {int(now.timestamp()* 1000)}] AND accountid:{gis.properties.id}", 
            max_items=-1, as_dict=True)
pd.DataFrame(search['results']).type.value_counts()

#### User Searches

In [ ]:
import string
last_login = set()
users = gis.users.advanced_search(f"accountid:{gis.properties.id}", max_users=-1)
[user for user in users['results'] if user['lastLogin'] == -1]

**Find Users Haven't Logged in for 90+ Days**

In [ ]:
now = _dt.datetime.now(_dt.timezone.utc)
greater_than90days = [user for user in users['results'] if user['lastLogin'] > -1 and \
 (now - _dt.datetime.fromtimestamp(user['lastLogin']/1000, _dt.timezone.utc)).days >= 90]
greater_than90days